In [87]:
#Loading packages

import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, roc_auc_score

In [88]:
#Loading data

df_Original = pd.read_csv('train.csv')

#Saving and dropping the dependant variable
#Y_SalePrice = df_Original.SalePrice
#df_Original = df_Original.drop(["SalePrice"], axis = 1)

#Displaying first 5 rows
df_Original.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [89]:
#Removing Id column

df_PreProcessed = df_Original.drop(["Id"], axis = 1)

df_Original.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [90]:
#Selecting categorical variable

FreqCountDict = {}

for column in df_PreProcessed:
    FreqCount = df_PreProcessed[column].value_counts()
    if len(FreqCount) < 50:
        FreqCountDict[column] = FreqCount

#All the categorical columns
print(FreqCountDict.keys(), "\n\n Number of categorical variables = ", len(FreqCountDict))


dict_keys(['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'LowQualFinSF', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageCars', 'GarageQual', 'GarageCond', 'PavedDrive', '3SsnPorch', 'PoolArea', 'PoolQC', 'Fence', 'MiscFeature', 'MiscVal', 'MoSold', 'YrSold', 'SaleType', 'SaleCondition']) 

 Number of categorical variables =  61


In [91]:
#Treating NAs in categorical variables before OneHotEncoding
#We first get the list of all categorical features with NAs as the data description has NA as a value in
#most categorical features

CategoricalFeatures_WithNAs = []

for cat_feature in list(FreqCountDict.keys()):
    if df_PreProcessed[cat_feature].isnull().any():
        CategoricalFeatures_WithNAs.append(cat_feature)

CategoricalFeatures_WithNAs

['Alley',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

In [101]:
#Looking at the "data_description.txt" file, only MasVnrType & Electrical features are truly NAs. NAs in the rest
#of the features are explained in file

print("Number of NAs in MasVnrType : ", df_PreProcessed['MasVnrType'].isnull().sum())
print("\nNumber of NAs in Electrical : ", df_PreProcessed['Electrical'].isnull().sum())

#Replacing NAs with relevant category

df_PreProcessed.loc[df_PreProcessed['MasVnrType'].isnull()] = "None"
df_PreProcessed.loc[df_PreProcessed['Electrical'].isnull()] = "SBrkr"

Number of NAs in MasVnrType :  8

Number of NAs in Electrical :  1


In [ ]:
#Treating NAs of numerical variables before OneHotEncoding adds columns to the data


In [84]:
#OneHotEncoding categorical features

for cat_feature in list(FreqCountDict.keys()):
    df_PreProcessed = df_PreProcessed.join(pd.get_dummies(df_Original[cat_feature], prefix = cat_feature, prefix_sep= "_"))
    df_PreProcessed = df_PreProcessed.drop([cat_feature], axis = 1)

print("Length of original DataFrame : ", len(df_Original.columns),
      "\n\nLength of DataFrame after OneHotEncoding : ", len(df_PreProcessed.columns))

ValueError: columns overlap but no suffix specified: Index(['MSSubClass_20', 'MSSubClass_30', 'MSSubClass_40', 'MSSubClass_45',
       'MSSubClass_50', 'MSSubClass_60', 'MSSubClass_70', 'MSSubClass_75',
       'MSSubClass_80', 'MSSubClass_85', 'MSSubClass_90', 'MSSubClass_120',
       'MSSubClass_160', 'MSSubClass_180', 'MSSubClass_190'],
      dtype='object')

In [81]:
#Treating NAs

for col in df_PreProcessed:
    

SyntaxError: unexpected EOF while parsing (<ipython-input-81-abf2e41da748>, line 4)

In [61]:
#Running linear regression to set baseline

#Splitting into test and train datasets on a 70/30 split
train = df_PreProcessed[:round(0.7*len(df_PreProcessed))]
test = df_PreProcessed[round(0.7*len(df_PreProcessed)) + 1:len(df_PreProcessed)]

#Removing dependent variable from test and train set
Y = train.SalePrice
X = train.drop(["SalePrice"], axis = 1)

Y_Truth = test.SalePrice
test = test.drop(["SalePrice"], axis = 1)

#Training the regression model
LinReg = LinearRegression()
LinReg.fit(X=X, y=Y)

LinReg.predict(X=test)



ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [100]:
df_PreProcessed.loc[df_PreProcessed["Electrical"].isnull()]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1379,80,RL,73.0,9735,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,5,2008,WD,Normal,167500
